In [1]:
%cd drive/MyDrive/Thesis\ Datasets
import csv
import string
import warnings
warnings.filterwarnings('ignore')

/content/drive/MyDrive/Thesis Datasets


In [2]:
import random
from collections import Counter
import numpy as np


In [3]:
!pip install pysentiment2

     |████████████████████████████████| 1.9 MB 41.0 MB/s 


In [ ]:
def create_sentence_data(dataset, step):
  names = []
  positions = []
  ethnicity = []
  total = 0
  ids = []
  id = 0
  reports = []
  header = ["ID", "Position", "Scouting Report Sentence(s)", "Race"]
  with open(dataset) as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
      id +=1
      sent = row[2].split(".")
      sent = sent[:-1]
      rem = len(sent) % step
      #print(sent)
      if rem != 0:
        upto = len(sent) - rem
        finisher = random.sample(sent[:upto], step-rem)
        sent.extend(finisher)
        #print(len(sent))
        #sent = sent[:len(sent)-rem]
      #print(len(sent))
      for i in range(0, len(sent)-1, step):
        ids.append(id)
        names.append(row[0])
        positions.append(row[1])
        #print(i, i+step)
        sentence = str(sent[i:i+step]).translate(str.maketrans('', '', '[]\''))
        reports.append(sentence)
        ethnicity.append(row[3])
  with open(str(step)+"_sentence_data_2021.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for i in range(0, len(names)):
      data = [ids[i], positions[i], reports[i], ethnicity[i]]
      writer.writerow(data)
        
create_sentence_data("dataset_2021_weka.csv", 1)

In [ ]:
create_sentence_data("dataset_2021_weka.csv", 2)      
create_sentence_data("dataset_2021_weka.csv", 5)

In [46]:
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
#from sklearn.pipeline import Pipeline
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imb_Pipeline, make_pipeline
from numpy import mean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
def get_accuracy_oversampling(dataset, stem):
  stem = True
  reports = []
  eth = []
  ps = PorterStemmer()
  stop_words = set(stopwords.words('english'))
  with open(dataset) as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
      # some preproccessing
      # removing stopwords
      if stem == True:
        word_tokens = word_tokenize(row[2])
        filtered = [w for w in word_tokens if not w.lower() in stop_words]
        filtered = []
        for w in word_tokens:
          if w not in stop_words:
            filtered.append(w)
        rep = " "
        rep = rep.join(filtered)

        # removing punctuation
        rep.translate(str.maketrans('', '', string.punctuation))
        # stemming
      
        reports.append(ps.stem(rep))
      else:
        reports.append(row[3])
      eth.append(row[3])

  rs = 3
 
  rep_train, rep_test, y_train, y_test = train_test_split(reports, eth, test_size = 0.25, random_state=rs)


  vectorizer = CountVectorizer()
  vectorizer.fit(rep_train)
  #X_train = vectorizer.transform(rep_train)
  #X_test = vectorizer.transform(rep_test)
  reports = vectorizer.transform(reports)
 
  # changine labels to integers for f1-score capabilities
  for i in range(0, len(eth)):
    if eth[i] == "W":
      eth[i] = 1
    else:
      eth[i] = 0
 

  # LR
  steps = [('over', RandomOverSampler()), ('model', LogisticRegression(random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=rs)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"LR f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"LR accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # NB
  steps = [('over', RandomOverSampler()), ('model', MultinomialNB())]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=3)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"NB f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"NB accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # SVM
  steps = [('over', RandomOverSampler()), ('model', svm.SVC(kernel='rbf', random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=3)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"SVM f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"SVM accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # SGD
  steps = [('over', RandomOverSampler()), ('model', SGDClassifier(random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"SGD f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"SGD accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

In [67]:
print("1 sentence accuracy...")
get_accuracy_oversampling("1_sentence_data_2021.csv", True)
print("\n2 sentence accuracy...")
get_accuracy_oversampling("2_sentence_data_2021.csv", True)
print("\n5 sentence accuracy...")
get_accuracy_oversampling("5_sentence_data_2021.csv", True)
print("\nAll sentence accuracy...")
get_accuracy_oversampling("dataset_2021_weka.csv", True)

1 sentence accuracy...
LR f1-score: 0.311.
LR accuracy: 0.711.
precision: 0.266.
recall: 0.377.

NB f1-score: 0.335.
NB accuracy: 0.679.
precision: 0.262.
recall: 0.465.

SVM f1-score: 0.179.
SVM accuracy: 0.807.
precision: 0.342.
recall: 0.123.

SGD f1-score: 0.297.
SGD accuracy: 0.706.
precision: 0.254.
recall: 0.358.


2 sentence accuracy...
LR f1-score: 0.334.
LR accuracy: 0.753.
precision: 0.316.
recall: 0.357.

NB f1-score: 0.399.
NB accuracy: 0.748.
precision: 0.342.
recall: 0.483.

SVM f1-score: 0.178.
SVM accuracy: 0.829.
precision: 0.537.
recall: 0.108.

SGD f1-score: 0.302.
SGD accuracy: 0.759.
precision: 0.305.
recall: 0.302.


5 sentence accuracy...
LR f1-score: 0.452.
LR accuracy: 0.817.
precision: 0.495.
recall: 0.427.

NB f1-score: 0.551.
NB accuracy: 0.837.
precision: 0.547.
recall: 0.563.

SVM f1-score: 0.252.
SVM accuracy: 0.841.
precision: 0.717.
recall: 0.157.

SGD f1-score: 0.418.
SGD accuracy: 0.814.
precision: 0.471.
recall: 0.382.


All sentence accuracy...
LR 

In [16]:
# read in GI as key-value pairs with keys being preprocessed (famili, confid, unnecessari, etc)
import csv
import pysentiment2 as ps
import re

def read_GI(filename):
  kvp = {}
  with open(filename ,'r', ) as file:
    csvreader = csv.reader(file)
    next(csvreader)
    next(csvreader)
    for col in csvreader:
      if col[1] != "Lvd": 
        kvp[col[0]] = [str(word) for word in col[2:-2] if word]
  
  kvp2 = {}
  hiv4 = ps.HIV4()   
  for key in kvp:
    new_key = str(hiv4.tokenize(key))
    # print(new_key)
    #print(bool(new_key))
    res = re.findall(r'\w+', new_key)
    # print(res)
    if res: 
      #print(res[0])
      kvp2[res[0]] = kvp[key].copy()
    else:
      kvp2[new_key] = kvp[key].copy()
  return kvp2

key_val = read_GI("inquirerbasic_py.csv")

In [14]:
# creating new csv's with GI sentiment for sentence(s)
def create_GI_sentiment_csv(dataset, gi_val):
  stop = 0
  tok_data = []
  names = []
  positions = []
  ethnicity = []
  total = 0
  ids = []
  id = 1
  header = ["ID", "Position", "Scouting Report Sentence(s)", "Race"]
  with open(dataset) as csv_file:
    csvreader = csv.reader(csv_file, delimiter=',')
    hiv4 = ps.HIV4()
    next(csvreader)
    for row in csvreader:
      tok_sent = []
      ids.append(row[0])
      names.append(row[0])
      positions.append(row[1])
      ethnicity.append(row[3])

      text = row[2]
      tokens = hiv4.tokenize(text)
      for tok in tokens:
        for words in gi_val:
          if tok == words:
            #print(f"{tok} senti is: {gi_val[words]}.")
            tok_sent.append(gi_val[words])
      x = [' '.join(y) for y in tok_sent]
      row_string = " ".join(x)
      tok_data.append(row_string)
    id+=1
  #print(tok_data)
  with open("gi_"+str(dataset), "w") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for i in range(0, len(names)):
      #print(ids[i], positions[i], tok_data[i], ethnicity[i])
      data = [ids[i], positions[i], tok_data[i], ethnicity[i]]
      writer.writerow(data)




In [ ]:
print(key_val["confid"])

['Positiv', 'Pstv', 'Strong', 'Power', 'Pleasur', 'EMOT', 'Ovrst', 'WlbPsyc', 'WlbTot']


In [17]:
create_GI_sentiment_csv("1_sentence_data_2021.csv", key_val)
create_GI_sentiment_csv("2_sentence_data_2021.csv", key_val)
create_GI_sentiment_csv("5_sentence_data_2021.csv", key_val)
create_GI_sentiment_csv("dataset_2021_weka.csv", key_val)

In [70]:
print("1 sentence accuracy for GI tagged reports...")
get_accuracy_oversampling("gi_1_sentence_data_2021.csv", False)
print("\n2 sentence accuracy for GI tagged reports...")
get_accuracy_oversampling("gi_2_sentence_data_2021.csv", False)
print("\n5 sentence accuracy for GI tagged reports...")
get_accuracy_oversampling("gi_5_sentence_data_2021.csv", False)
print("\nAll sentence accuracy for GI tagged reports...")
get_accuracy_oversampling("gi_dataset_2021_weka.csv", False)

1 sentence accuracy for GI tagged reports...
LR f1-score: 0.303.
LR accuracy: 0.571.
precision: 0.211.
recall: 0.535.

NB f1-score: 0.295.
NB accuracy: 0.581.
precision: 0.209.
recall: 0.503.

SVM f1-score: 0.273.
SVM accuracy: 0.672.
precision: 0.223.
recall: 0.353.

SGD f1-score: 0.292.
SGD accuracy: 0.575.
precision: 0.208.
recall: 0.506.


2 sentence accuracy for GI tagged reports...
LR f1-score: 0.318.
LR accuracy: 0.613.
precision: 0.229.
recall: 0.521.

NB f1-score: 0.308.
NB accuracy: 0.608.
precision: 0.222.
recall: 0.502.

SVM f1-score: 0.308.
SVM accuracy: 0.699.
precision: 0.257.
recall: 0.387.

SGD f1-score: 0.306.
SGD accuracy: 0.578.
precision: 0.221.
recall: 0.55.


5 sentence accuracy for GI tagged reports...
LR f1-score: 0.389.
LR accuracy: 0.709.
precision: 0.311.
recall: 0.526.

NB f1-score: 0.364.
NB accuracy: 0.668.
precision: 0.276.
recall: 0.541.

SVM f1-score: 0.365.
SVM accuracy: 0.713.
precision: 0.302.
recall: 0.468.

SGD f1-score: 0.339.
SGD accuracy: 0.664

In [ ]:
def create_gi_dataset(filename, gi_tags):
  senti = []
  eth = []
  sent_gi = []
  with open(gi_tags) as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
      senti.append(row[0])


  

  
  with open(filename) as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
      sent_gi.append(row[2])
      eth.append(row[3])

  data_rows = []
  
  #print(sent_gi[433])
  for i in range(len(sent_gi)):
    senti_tmp = senti.copy()
    index = 0
    for word in senti:
      if word in sent_gi[i]:
        senti_tmp[index] = 1
      else:
        senti_tmp[index] = 0
      index +=1
    data_rows.append(senti_tmp)
  

  for i in range(0, len(eth)):
    data_rows[i].append(eth[i])
  
  print(len(data_rows[0]))
  header = senti +["Race"]
  with open("gi_tagged_"+str(filename), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data_rows)
  
  



In [ ]:
create_gi_dataset("gi_1_sentence_data_2021.csv", "Enquirer_comparison_v2.csv")
create_gi_dataset("gi_2_sentence_data_2021.csv", "Enquirer_comparison_v2.csv")
create_gi_dataset("gi_5_sentence_data_2021.csv", "Enquirer_comparison_v2.csv")
create_gi_dataset("gi_dataset_2021_weka.csv", "Enquirer_comparison_v2.csv")

157
157
157
157


In [71]:
def classification_gi(filename):
  eth = []
  reports = []
  with open(filename) as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
      eth.append(row[156])
      for i in range(0, len(row)-1):
        row[i] = int(row[i])
      reports.append(row[:156])

  rs = 3
  # changine labels to integers for f1-score capabilities
  for i in range(0, len(eth)):
    if eth[i] == "W":
      eth[i] = 1
    else:
      eth[i] = 0
  #rep_train, rep_test, y_train, y_test = train_test_split(tags, eth, test_size = 0.25)
  #clg = LogisticRegression().fit(rep_train, y_train)
  #score = clg.score(rep_test, y_test)
  #print(score)

# LR
  steps = [('over', RandomOverSampler()), ('model', LogisticRegression(random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=rs)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"LR f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"LR accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # NB
  steps = [('over', RandomOverSampler()), ('model', MultinomialNB())]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=3)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"NB f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"NB accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # SVM
  steps = [('over', RandomOverSampler()), ('model', svm.SVC(kernel='rbf', random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=3)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"SVM f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"SVM accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")

  # SGD
  steps = [('over', RandomOverSampler()), ('model', SGDClassifier(random_state=rs))]
  pipeline = imb_Pipeline(steps=steps)
  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
  scores = cross_validate(pipeline, reports, eth, cv=cv, scoring=('f1', 'precision', 'recall', 'accuracy'))
  score = mean(scores['test_f1'])
  print(f"SGD f1-score: {round(score, 3)}.")
  score = mean(scores['test_accuracy'])
  print(f"SGD accuracy: {round(score, 3)}.")
  score = mean(scores['test_precision'])
  print(f"precision: {round(score, 3)}.")
  score = mean(scores['test_recall'])
  print(f"recall: {round(score, 3)}.\n")



In [72]:
print("1 sentence accuracy for GI tagged reports...")
classification_gi("gi_tagged_gi_1_sentence_data_2021.csv")
print("\n2 sentence accuracy for GI tagged reports...")
classification_gi("gi_tagged_gi_2_sentence_data_2021.csv")
print("\n5 sentence accuracy for GI tagged reports...")
classification_gi("gi_tagged_gi_5_sentence_data_2021.csv")
print("\nAll sentence accuracy for GI tagged reports...")
classification_gi("gi_tagged_gi_dataset_2021_weka.csv")

1 sentence accuracy for GI tagged reports...
LR f1-score: 0.304.
LR accuracy: 0.573.
precision: 0.212.
recall: 0.535.

NB f1-score: 0.296.
NB accuracy: 0.579.
precision: 0.209.
recall: 0.508.

SVM f1-score: 0.249.
SVM accuracy: 0.679.
precision: 0.211.
recall: 0.306.

SGD f1-score: 0.286.
SGD accuracy: 0.539.
precision: 0.197.
recall: 0.539.


2 sentence accuracy for GI tagged reports...
LR f1-score: 0.314.
LR accuracy: 0.602.
precision: 0.224.
recall: 0.527.

NB f1-score: 0.312.
NB accuracy: 0.607.
precision: 0.225.
recall: 0.514.

SVM f1-score: 0.254.
SVM accuracy: 0.735.
precision: 0.25.
recall: 0.261.

SGD f1-score: 0.273.
SGD accuracy: 0.611.
precision: 0.214.
recall: 0.442.


5 sentence accuracy for GI tagged reports...
LR f1-score: 0.312.
LR accuracy: 0.649.
precision: 0.24.
recall: 0.455.

NB f1-score: 0.328.
NB accuracy: 0.645.
precision: 0.248.
recall: 0.489.

SVM f1-score: 0.253.
SVM accuracy: 0.754.
precision: 0.281.
recall: 0.235.

SGD f1-score: 0.309.
SGD accuracy: 0.617.